# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
# Inicializando Spark
import findspark
findspark.init('/home/macaubas/spark-3.2.1-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName("logisticRegExe").getOrCreate()

22/05/16 16:34:06 WARN Utils: Your hostname, macaubas-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/05/16 16:34:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/16 16:34:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/16 16:34:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Regressão linear
from pyspark.ml.classification import LogisticRegression

# Features
from pyspark.ml.feature import (VectorAssembler, VectorIndexer,
                               OneHotEncoder, StringIndexer)

# Pipeline
from pyspark.ml import Pipeline

# Evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [79]:
# Load data
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

# Esquema
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [114]:
## Descrevendo
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

Não há dados ausentes, todos as colunas possuem 900 observações

In [120]:
# Viz
for e in df.head(5):
    print(e)
    print('\n')

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date='2013-08-30 07:00:40', Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)


Row(Names='Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date='2013-08-13 00:38:46', Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company='Wilson PLC', Churn=1)


Row(Names='Eric Lozano', Age=38.0, Total_Purchase=12884.75, Account_Manager=0, Years=6.67, Num_Sites=12.0, Onboard_date='2016-06-29 06:20:07', Location='1331 Keith Court Alyssahaven, DE 90114', Company='Miller, Johnson and Wallace', Churn=1)


Row(Names='Phillip White', Age=42.0, Total_Purchase=8010.76, Account_Manager=0, Years=6.71, Num_Sites=10.0, Onboard_date='2014-04-22 12:43:12', Location='13120 Daniel Mount Angelabury, WY 30645-4695', Company='Smith Inc', Churn=1)


Row(Names='Cynthia Norton', Age=37.0, Total_Pu

## Selecionando variáveis de interesse

In [121]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [122]:
# Selecionando colunas de interesse
my_cols = df.select(['Age', 'Total_Purchase',
                     'Account_Manager','Years','Num_Sites',
                     'Churn'])

In [129]:
# Viz
for e in my_cols.head(5):
    print(e)
    print('\n')

Row(Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Churn=1)


Row(Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Churn=1)


Row(Age=38.0, Total_Purchase=12884.75, Account_Manager=0, Years=6.67, Num_Sites=12.0, Churn=1)


Row(Age=42.0, Total_Purchase=8010.76, Account_Manager=0, Years=6.71, Num_Sites=10.0, Churn=1)


Row(Age=37.0, Total_Purchase=9191.58, Account_Manager=0, Years=5.56, Num_Sites=9.0, Churn=1)




## Assembling

In [131]:
## Criando assembler
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase',
                                       'Account_Manager','Years','Num_Sites'], 
                            outputCol='features')

output = assembler.transform(df)

# selecionando
final_data = output.select('features', 'Churn')

## Modelo

In [133]:
# Definindo Modelo
log_reg_churn = LogisticRegression(featuresCol = 'features', labelCol = 'Churn')

## Teste e treino

In [134]:
# Datasets de treino e teste
treino,teste = final_data.randomSplit([0.7,0.3])

In [135]:
## Fitted model
fitted_model = log_reg_churn.fit(treino)

## Sumarização do modelo

In [136]:
## Sumarização
treino_summary = fitted_model.summary

In [138]:
treino_summary.predictions.describe().show()

+-------+-------------------+------------------+
|summary|              Churn|        prediction|
+-------+-------------------+------------------+
|  count|                625|               625|
|   mean|             0.1696|            0.1296|
| stddev|0.37558160027021886|0.3361318422652269|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



## Avaliando o modelo

In [149]:
# Previsão e labels
pred_e_labels = fitted_model.evaluate(teste)

pred_e_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.59178722784752...|[0.98996695310486...|       0.0|
|[25.0,9672.03,0.0...|    0|[4.80566275265393...|[0.99188314671939...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.49571021229584...|[0.99849237388731...|       0.0|
|[28.0,8670.98,0.0...|    0|[7.86749926813158...|[0.99961715570263...|       0.0|
|[28.0,11128.95,1....|    0|[4.10572580665544...|[0.98378906997499...|       0.0|
|[28.0,11245.38,0....|    0|[3.83058991066019...|[0.97876394209739...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.54796159365946...|[0.98952218057536...|       0.0|
|[29.0,10203.18,1....|    0|[3.61215838249485...|[0.97371597674899...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.19186546503934...|[0.96052701011367...|       0.0|
|[30.0,10960.52,

In [141]:
# rawPredictionCol tem o nome de prediction pq por default quando damos fit no modelo, sua coluna de predição tem esse nome
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', 
                                        labelCol='Churn')

In [155]:
auc = my_eval.evaluate(pred_e_labels.predictions)

auc

0.7348484848484849

## Deploying Modelo

In [161]:
final_log_reg_model = log_reg_churn.fit(final_data)

In [162]:
unlabeled_data = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

unlabeled_data.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [163]:
## Esquema
unlabeled_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [165]:
# Criando coluna de features
teste_novos_consumidores = assembler.transform(unlabeled_data)

teste_novos_consumidores.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [166]:
# Transformando
resultados = final_log_reg_model.transform(teste_novos_consumidores)

In [168]:
# Vizualindo
resultados.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



## Conclusão

As empresas Cannon-Benson, Barron-Robertson, Sexton-Golden e Parks-Robbins são provavéis de se desligar e a sugestão é que a empresa direcione gerentes para tentar retê-los.